# Mapping des Équipes - World Cup ETL

**Auteur** : Short Kings Team  
**Date** : 16/12/2025

## Objectif
Créer le référentiel des équipes nationales avec leurs aliases pour harmoniser les noms à travers tous les datasets.

## Architecture

Les constantes et fonctions de mapping sont centralisées dans des modules Python :

- **`src/teams_constants.py`** : Constantes (ALIASES_MAPPING, HISTORICAL_TEAMS, PLACEHOLDERS)
- **`src/teams_reference.py`** : Fonctions de normalisation et référentiel
- **`src/normalize_teams.py`** : Script principal de normalisation

Ce notebook sert de **documentation** et d'**exploration** des données.

## 1. Imports

In [1]:
import sys
sys.path.append('../src')

from teams_constants import ALIASES_MAPPING, HISTORICAL_TEAMS, PLACEHOLDERS
from teams_reference import (
    normalize_team_name,
    get_confederation,
    get_aliases,
    is_historical_team,
    build_teams_reference,
    CONFEDERATIONS
)
from normalize_teams import normalize_teams

print(f"Confédérations chargées : {len(CONFEDERATIONS)}")
print(f"Aliases définis : {len(ALIASES_MAPPING)}")
print(f"Équipes historiques : {len(HISTORICAL_TEAMS)}")

Confédérations chargées : 231
Aliases définis : 170
Équipes historiques : 18


## 2. Documentation des constantes

### Équipes historiques

Ces équipes n'existent plus dans le classement FIFA actuel mais ont participé à des Coupes du Monde.

**Source** : [FIFA World Cup Archives](https://www.fifa.com/tournaments/mens/worldcup)

**Règle appliquée** : Conformément aux règles FIFA/UEFA, chaque équipe historique reste une entité distincte pour l'attribution des matchs.

In [2]:
print("Équipes historiques (entités distinctes) :")
print("-" * 60)
for team, (year, successor, conf) in HISTORICAL_TEAMS.items():
    successor_str = f"→ {successor}" if successor else "(pas de successeur)"
    print(f"  {team} ({conf}) - dissous en {year} {successor_str}")

Équipes historiques (entités distinctes) :
------------------------------------------------------------
  Soviet Union (UEFA) - dissous en 1991 → Russia
  Yugoslavia (UEFA) - dissous en 2003 → Serbia
  Czechoslovakia (UEFA) - dissous en 1993 → Czech Republic
  Serbia-Montenegro (UEFA) - dissous en 2006 → Serbia
  GDR (UEFA) - dissous en 1990 → Germany
  Saarland (UEFA) - dissous en 1956 → Germany
  Irish Free State (UEFA) - dissous en 1936 → Republic of Ireland
  Zaire (CAF) - dissous en 1997 → Congo DR
  Dahomey (CAF) - dissous en 1975 → Benin
  Upper Volta (CAF) - dissous en 1984 → Burkina Faso
  Rhodesia (CAF) - dissous en 1980 → Zimbabwe
  Dutch East Indies (AFC) - dissous en 1945 → Indonesia
  North Yemen (AFC) - dissous en 1990 → Yemen
  South Yemen (AFC) - dissous en 1990 → Yemen
  South Vietnam (AFC) - dissous en 1976 → Vietnam
  Dutch Guyana (CONMEBOL) - dissous en 1975 → Suriname
  Dutch Antilles (CONCACAF) - dissous en 2010 (pas de successeur)
  Western Samoa (OFC) - dissous

## 3. Tests des fonctions de normalisation

In [3]:
test_cases = [
    "Brazil (Brasil)",
    "Soviet Union (СССР)",
    "ARGENTINA",
    "A1",  # Placeholder
    "Germany",
    "IR Iran",
    "FRG",  # West Germany -> Germany
    "Ivory Coast (Côte d'Ivoire)",
]

print("Tests de normalisation :")
print("-" * 70)
for test in test_cases:
    normalized = normalize_team_name(test)
    conf = get_confederation(test)
    is_hist = is_historical_team(normalized) if normalized else False
    print(f"'{test}'")
    print(f"  → '{normalized}' | Conf: {conf} | Historique: {is_hist}")

Tests de normalisation :
----------------------------------------------------------------------
'Brazil (Brasil)'
  → 'Brazil' | Conf: CONMEBOL | Historique: False
'Soviet Union (СССР)'
  → 'Soviet Union' | Conf: UEFA | Historique: True
'ARGENTINA'
  → 'Argentina' | Conf: CONMEBOL | Historique: False
'A1'
  → 'None' | Conf: None | Historique: False
'Germany'
  → 'Germany' | Conf: UEFA | Historique: False
'IR Iran'
  → 'IR Iran' | Conf: AFC | Historique: False
'FRG'
  → 'Germany' | Conf: UEFA | Historique: False
'Ivory Coast (Côte d'Ivoire)'
  → 'Côte d'Ivoire' | Conf: CAF | Historique: False


## 4. Statistiques du référentiel

In [4]:
teams_ref = build_teams_reference()

print(f"Total équipes : {len(teams_ref)}")
print(f"Équipes historiques : {sum(1 for t in teams_ref.values() if t['is_historical'])}")
print(f"Équipes avec aliases : {sum(1 for t in teams_ref.values() if t['aliases'])}")

# Répartition par confédération
print("\nRépartition par confédération :")
conf_counts = {}
for data in teams_ref.values():
    conf = data['confederation']
    conf_counts[conf] = conf_counts.get(conf, 0) + 1

for conf, count in sorted(conf_counts.items(), key=lambda x: -x[1]):
    print(f"  {conf}: {count} équipes")

Total équipes : 231
Équipes historiques : 18
Équipes avec aliases : 145

Répartition par confédération :
  UEFA: 62 équipes
  CAF: 58 équipes
  AFC: 50 équipes
  CONCACAF: 37 équipes
  OFC: 13 équipes
  CONMEBOL: 11 équipes


## 5. Exécution de la normalisation

In [5]:
# Exécuter la normalisation complète
df_result, unmatched = normalize_teams()


FUSION DES ÉQUIPES EN DOUBLON

Algeria:
  IDs trouvés: [3, 4]
  ID conservé: 3
  IDs à remapper: [4] → 3

Antigua and Barbuda:
  IDs trouvés: [9, 10]
  ID conservé: 9
  IDs à remapper: [10] → 9

Belgium:
  IDs trouvés: [22, 23]
  ID conservé: 22
  IDs à remapper: [23] → 22

Bosnia and Herzegovina:
  IDs trouvés: [28, 29]
  ID conservé: 28
  IDs à remapper: [29] → 28

Brazil:
  IDs trouvés: [31, 32]
  ID conservé: 31
  IDs à remapper: [32] → 31

Cameroon:
  IDs trouvés: [39, 40]
  ID conservé: 39
  IDs à remapper: [40] → 39

Croatia:
  IDs trouvés: [53, 54]
  ID conservé: 53
  IDs à remapper: [54] → 53

Denmark:
  IDs trouvés: [62, 63]
  ID conservé: 62
  IDs à remapper: [63] → 62

Egypt:
  IDs trouvés: [72, 73]
  ID conservé: 72
  IDs à remapper: [73] → 72

Germany:
  IDs trouvés: [80, 89, 90]
  ID conservé: 80
  IDs à remapper: [89, 90] → 80

Greece:
  IDs trouvés: [92, 93]
  ID conservé: 92
  IDs à remapper: [93] → 92

Guyana:
  IDs trouvés: [97, 100]
  ID conservé: 97
  IDs à remap

In [6]:
# Aperçu du résultat
df_result.head(10)

,id_team,nom_standard,confederation,aliases
0,1,Afghanistan,AFC,"[""Afghanistan (افغانستان)""]"
1,2,Albania,UEFA,"[""Albania (Shqipëri)""]"
2,3,Algeria,CAF,"[""Algeria (الجزائر)""]"
3,5,American Samoa,OFC,[]
4,6,Andorra,UEFA,[]
5,7,Angola,CAF,[]
6,8,Anguilla,CONCACAF,[]
7,9,Antigua and Barbuda,CONCACAF,"[""Antigua""]"
8,11,Argentina,CONMEBOL,[]
9,12,Armenia,UEFA,"[""Armenia (Հdelays)""]"
